In [26]:
#Importing the necessary modules
from wise_light_curves.wise_light_curves import *
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle 
from matplotlib.patches import ConnectionPatch
import numpy as np
import pandas as pd
import os
import scipy
from Potato import plotparams
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
#End of Importing

In [27]:
#This is where the asassnid goes
id_val = 352187470767																																																																													

ra_val = 134.564850   			

dec_val = -43.165320													

cust_id = 'J085816-430955'

#																														

In [28]:
def read_lightcurve_csv(asas_sn_id, guide = 'known_dipper_lightcurves/'):
    """
    Input: asas_sn_id of desired star

    Output: Two dataframes. One contains the V-band data for the star and the other contains the g-band data for the star.

    This function reads the data of the desired star by going to the corresponding file and copying the data of that file onto a
    data frame. This data frame is then sorted into two data frames by comparing the value in the Photo filter column. If the Photo 
    filter column data has a value of one, its row is sorted into the dataframe corresponding to the V-band. If the Photo filter column 
    data has a value of zero, it gets sorted into the dataframe corresponding to the g-band.
    """
    
    fname = os.path.join(guide, str(asas_sn_id)+'.csv')

    dfv = pd.DataFrame()
    dfg = pd.DataFrame()

    fdata = pd.read_csv(fname, header=None)
    fdata.columns = ["JD", "Flux", "Flux Error", "Mag", "Mag Error", "Limit", "FWHM", "Filter", "Quality", "Camera"]

    dfv = fdata.loc[fdata["Filter"] == 'V'].reset_index(drop=True)
    dfg = fdata.loc[fdata["Filter"] == 'g'].reset_index(drop=True)

    return dfv, dfg
#End of read_lightcurve_csv function

In [29]:
def read_lightcurve_csv_alt(asas_sn_id, guide = 'known_dipper_lightcurves/'):
    """
    Input: asas_sn_id of desired star

    Output: Two dataframes. One contains the V-band data for the star and the other contains the g-band data for the star.

    This function reads the data of the desired star by going to the corresponding file and copying the data of that file onto a
    data frame. This data frame is then sorted into two data frames by comparing the value in the Photo filter column. If the Photo 
    filter column data has a value of one, its row is sorted into the dataframe corresponding to the V-band. If the Photo filter column 
    data has a value of zero, it gets sorted into the dataframe corresponding to the g-band.
    """
    
    fname = os.path.join(guide, str(asas_sn_id)+'.csv')

    dfv = pd.DataFrame()
    dfg = pd.DataFrame()

    fdata = pd.read_csv(fname, header=None)
    fdata.columns = ["JD", "UT Date", "Camera", "FWHM", "Limit", "Mag", "Mag_err", "Flux", "Flux_err", "Filter"]

    dfv = fdata.loc[fdata["Filter"] == 'V'].reset_index(drop=True)
    dfg = fdata.loc[fdata["Filter"] == 'g'].reset_index(drop=True)

    return dfv, dfg
#End of read_lightcurve_csv function

In [30]:
dfv, dfg = read_lightcurve_csv_alt(id_val, 'Updated_LC_data')

dfg['Mag'] = [float(i) for i in dfg['Mag']]

dfg['JD'] = [float(i) for i in dfg['JD']]

dfv['Mag'] = [float(i) for i in dfv['Mag']]

dfv['JD'] = [float(i) for i in dfv['JD']]

dfg = dfg.loc[dfg['Mag'] < 17].reset_index(drop=True)

mag = dfg['Mag']

jd = dfg['JD']

meanmag = sum(mag) / len(mag)

dfg_mag_avg = [i - meanmag for i in mag]

peaks = scipy.signal.find_peaks(dfg_mag_avg, prominence=0.17, distance=25, height=0.3, width=2) 

peak = peaks[0]

prop = peaks[1]

length = len(peak)

peak = [int(i) for i in peak]

peak = pd.Series(peak)

In [31]:
dfv.head()

,JD,UT Date,Camera,FWHM,Limit,Mag,Mag_err,Flux,Flux_err,Filter
0,2.457421e+06,2016-02-02.0402217,bg,1.80,15.441,11.942,0.010,64.089,0.545,V
1,2.457421e+06,2016-02-02.0415115,bg,1.89,15.485,11.962,0.008,62.970,0.514,V
2,2.457421e+06,2016-02-02.0427909,bg,1.83,15.452,11.945,0.009,63.955,0.534,V
3,2.457421e+06,2016-02-02.0480015,bh,1.57,15.545,11.965,0.008,62.784,0.475,V
4,2.457421e+06,2016-02-02.0492883,bh,1.52,15.552,11.965,0.008,62.767,0.474,V


In [32]:
dfg = dfg.loc[dfg['Mag'] < 17].reset_index(drop=True)
dfg.head()

,JD,UT Date,Camera,FWHM,Limit,Mag,Mag_err,Flux,Flux_err,Filter
0,2.458072e+06,2017-11-14.3509530,bl,1.45,16.044,12.305,0.007,43.489,0.284,g
1,2.458072e+06,2017-11-14.3521799,bl,1.48,16.044,12.299,0.007,43.686,0.286,g
2,2.458072e+06,2017-11-14.3534101,bl,1.47,16.023,12.295,0.007,43.843,0.293,g
3,2.458073e+06,2017-11-15.3502568,bl,1.52,16.056,12.295,0.007,43.887,0.282,g
4,2.458073e+06,2017-11-15.3514765,bl,1.51,16.036,12.302,0.007,43.585,0.285,g


In [33]:
print('The mean magnitude of the whole light curve:', meanmag)

The mean magnitude of the whole light curve: 12.304846162790698


In [ ]:
c = SkyCoord(ra=ra_val*u.degree, dec=dec_val*u.degree, frame='icrs')
print(c.ra.hms)
print(c.dec.dms)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax = plotparams(ax)
ax.set_title(cust_id)

top = 13.5
bottom = 12.9

cams = dfg["Camera"]
camtype = np.unique(cams)
camnum = len(camtype)

colors = ["#6b8bcd", "#b3b540", "#8f62ca", "#5eb550", "#c75d9c", "#4bb092", "#c5562f", "#6c7f39", 
              "#ce5761", "#c68c45", '#b5b246', '#d77fcc', '#7362cf', '#ce443f', '#3fc1bf', '#cda735',
              '#a1b055']
    
camdf = pd.DataFrame()

for i in range(0,camnum):
    camdf = dfg.loc[dfg["Camera"] == camtype[i]].reset_index(drop=True)
    for j in range(0,len(camdf)):
        camjd = [float(i) for i in camdf["JD"]]
        cammag = [float(i) for i in camdf['Mag']]
        ax.scatter((camjd[j] - (2.458 * 10 **6)), cammag[j], color = colors[i], alpha = 0.6)

#for i in range(len(peak)):
    #ax.vlines((jd[peak[i]] - (2.458 * 10**6)), bottom, top, "k", alpha=0.3)

#ax.set_ylim(bottom,top)
ax.invert_yaxis() 
ax.set_xlabel('Julian Date - $2458000$ [d]', fontsize=20)
ax.set_ylabel('g [mag]', fontsize=20)
plt.show() 

In [ ]:
left = 2600
right = 2800

dfg_dip = dfg.loc[dfg["JD"] > (left + 2.458 * 10**6)].reset_index(drop=True)
dfg_dip = dfg_dip.loc[dfg_dip["JD"] < (right + 2.458 * 10**6)].reset_index(drop=True)
print(len(dfg_dip))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 4))
fig.suptitle(cust_id, fontsize = 22)

ax[0] = plotparams(ax[0])

cams = dfg["Camera"]
camtype = np.unique(cams)
camnum = len(camtype)

colors = ["#6b8bcd", "#b3b540", "#8f62ca", "#5eb550", "#c75d9c", "#4bb092", "#c5562f", "#6c7f39", 
              "#ce5761", "#c68c45", '#b5b246', '#d77fcc', '#7362cf', '#ce443f', '#3fc1bf', '#cda735',
              '#a1b055']
    
camdf = pd.DataFrame()

for i in range(0,camnum):
    camdf = dfg.loc[dfg["Camera"] == camtype[i]].reset_index(drop=True)
    for j in range(0,len(camdf)):
        camjd = [float(i) for i in camdf["JD"]]
        cammag = [float(i) for i in camdf['Mag']]
        ax[0].scatter((camjd[j] - (2.458 * 10 **6)), cammag[j], color = colors[i], alpha = 0.6)

#ax[0].scatter((dfg["JD"] - (2.458 * 10 **6)), dfg['Mag'], color = "#6b8bcd", alpha = 0.6)
#ax[0].add_patch(Rectangle(
 #   (left,(bottom + 0.05)),
  #  (right-left),
   # (top-bottom-0.075),
    #fc ='none',  
    #ec ='black', 
    #lw = 2))
#ax[0].set_ylim(bottom,top)
ax[0].invert_yaxis() 
ax[0].set_xlabel('Julian Date - $2458000$ [d]', fontsize=20)
ax[0].set_ylabel('g [mag]', fontsize=20)

ax[1] = plotparams(ax[1])

cams2 = dfg_dip["Camera"]
camtype2 = np.unique(cams2)
camnum2 = len(camtype2)

colors2 = ["#6b8bcd", "#b3b540", "#8f62ca", "#5eb550", "#c75d9c", "#4bb092", "#c5562f", "#6c7f39", 
              "#ce5761", "#c68c45", '#b5b246', '#d77fcc', '#7362cf', '#ce443f', '#3fc1bf', '#cda735',
              '#a1b055']
    
camdf2 = pd.DataFrame()

for i in range(0,camnum2):
    camdf2 = dfg_dip.loc[dfg_dip["Camera"] == camtype2[i]].reset_index(drop=True)
    for j in range(0,len(camdf2)):
        camjd2 = [float(i) for i in camdf2["JD"]]
        cammag2 = [float(i) for i in camdf2['Mag']]
        ax[1].scatter((camjd2[j] - (2.458 * 10 **6)), cammag2[j], color = colors2[i], alpha = 0.6)

#ax[1].scatter((dfg_dip["JD"] - (2.458 * 10 **6)), dfg_dip['Mag'], color = "#6b8bcd", alpha = 0.6)
#ax[1].set_ylim(bottom,top)
ax[1].invert_yaxis() 
ax[1].set_xlabel('Julian Date - $2458000$ [d]', fontsize=20)
#ax[1].set_ylabel('g [mag]', fontsize=20)

offset = -20

con1 = ConnectionPatch(xyA =(right, (top - 0.025)),
                      coordsA = ax[0].transData,
                      xyB =((left+offset), top), 
                      coordsB = ax[1].transData)

con1.set_color([0, 0, 0])
con1.set_linewidth(2)
#ax[1].add_artist(con1)

con2 = ConnectionPatch(xyA =(right, (bottom +0.05)),
                      coordsA = ax[0].transData,
                      xyB =((left+offset), bottom), 
                      coordsB = ax[1].transData)

con2.set_color([0, 0, 0])
con2.set_linewidth(2)
#ax[1].add_artist(con2)

plt.show() 

In [ ]:
dfg_nodip1 = dfg.loc[dfg["JD"] < (left + 2.458 * 10**6)].reset_index(drop=True)
dfg_nodip2 = dfg.loc[dfg["JD"] > (right + 2.458 * 10**6)].reset_index(drop=True)
dfg_nodip = pd.concat([dfg_nodip1,dfg_nodip2])
print(len(dfg_nodip))
print(len(dfg_dip))
print(len(dfg))

In [ ]:
meannodip = np.mean(dfg_nodip['Mag'])
stdnodip = np.std(dfg_nodip['Mag'])

deep = max(dfg_dip['Mag'])

print('The mean of the LC w/out the dip:', meannodip)
print('The std of the LC w/out the dip:',stdnodip)
print('The deepest point of the dip:', deep)

In [ ]:
dfg_time = dfg_dip.loc[dfg_dip['Mag'] >= (meannodip + 2*stdnodip)].reset_index(drop=True)

t1 = dfg_time['JD'][0]
t2 = dfg_time['JD'][len(dfg_time['JD'])-1]

time1 = Time(t1, format='jd')
time1.format = 'iso'

time2 = Time(t2, format='jd')
time2.format = 'iso'

print(time1)
print(time2)

fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax = plotparams(ax)
ax.set_title(cust_id)

cams = dfg_time["Camera"]
camtype = np.unique(cams)
camnum = len(camtype)

colors = ["#6b8bcd", "#b3b540", "#8f62ca", "#5eb550", "#c75d9c", "#4bb092", "#c5562f", "#6c7f39", 
              "#ce5761", "#c68c45", '#b5b246', '#d77fcc', '#7362cf', '#ce443f', '#3fc1bf', '#cda735',
              '#a1b055']
    
camdf = pd.DataFrame()

for i in range(0,camnum):
    camdf = dfg_time.loc[dfg_time["Camera"] == camtype[i]].reset_index(drop=True)
    for j in range(0,len(camdf)):
        camjd = [float(i) for i in camdf["JD"]]
        cammag = [float(i) for i in camdf['Mag']]
        ax.scatter((camjd[j] - (2.458 * 10 **6)), cammag[j], color = colors[i], alpha = 0.6)

#ax.scatter(dfg_time['JD'], dfg_time['Mag'], color = "#6b8bcd", alpha = 0.6)
ax.set_ylim(bottom,top)
ax.invert_yaxis() 
ax.set_xlabel('Julian Date - $2.458x10^{6}$ [d]', fontsize=20)
ax.set_ylabel('g [mag]', fontsize=20)
plt.show() 

In [ ]:
print(t2-t1)

In [ ]:
download_single_data(name='wise_'+str(cust_id), ra=ra_val, dec=dec_val, root_path='WISE_Data/', radius=2.)

allwise = ascii.read('WISE_Data/wise_'+str(cust_id)+'_allwise.ipac', format='ipac')
neowise = ascii.read('WISE_Data/wise_'+str(cust_id)+'_neowise.ipac', format='ipac')
allwise, neowise = only_good_data_v1(allwise, neowise)

print(allwise)
print(neowise)

# MJD = JD - 2400000.5.
allwise['jd'] = allwise['mjd'] + 2400000.5
neowise['jd'] = neowise['mjd'] + 2400000.5

alldf = pd.DataFrame(np.array(allwise))
neodf = pd.DataFrame(np.array(neowise))

In [ ]:
print(alldf.columns)
alldf.head(10)

In [ ]:
print(neodf.columns)
neodf.head(10)

In [ ]:
plot_full_lightcurve(allwise,neowise)

In [ ]:
alldf['bins'] = pd.cut(alldf['mjd'],2)

bins = alldf['bins'].unique()

all_df = pd.DataFrame()

for i in bins:
    allbin = alldf.loc[alldf['bins'] == i].reset_index(drop=True)
    allbin['w1mpro_ep'] = allbin['w1mpro_ep'].median()
    allbin['w2mpro_ep'] = allbin['w2mpro_ep'].median()
    all_df = pd.concat([all_df,allbin])

all_df.head(10)

In [ ]:
neodf['bins'] = pd.cut(neodf['mjd'],21)

bins = neodf['bins'].unique()

neo_df = pd.DataFrame()

for i in bins:
    neobin = neodf.loc[neodf['bins'] == i].reset_index(drop=True)
    neobin['w1mpro'] = neobin['w1mpro'].median()
    neobin['w2mpro'] = neobin['w2mpro'].median()
    neo_df = pd.concat([neo_df,neobin])

neo_df.head(10)

In [ ]:
wise_day1 = list(all_df['jd'])
wise_day2 = list(neo_df['jd'])
wise_day = wise_day1 + wise_day2

wise_w1_1 = list(all_df['w1mpro_ep'])
wise_w1_2 = list(neo_df['w1mpro'])
wise_w1 = wise_w1_1 + wise_w1_2

wise_w2_1 = list(all_df['w2mpro_ep'])
wise_w2_2 = list(neo_df['w2mpro'])
wise_w2 = wise_w2_1 + wise_w2_2

print(type(wise_day[0]))

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(8, 8))
fig.subplots_adjust(hspace=0)
ax[0] = plotparams(ax[0])
ax[0].set_title(cust_id)

wise_day_plot = [i - (2.458 * 10 **6) for i in wise_day]
dfg_day_plot = [i - (2.458 * 10 **6) for i in dfg['JD']]
dfv_day_plot = [i - (2.458 * 10 **6) for i in dfv['JD']]

colors = ["#6b8bcd", "#b3b540", "#8f62ca", "#5eb550", "#c75d9c", "#4bb092", "#c5562f", "#6c7f39", 
              "#ce5761", "#c68c45", '#b5b246', '#d77fcc', '#7362cf', '#ce443f', '#3fc1bf', '#cda735',
              '#a1b055']
    
ax[0].scatter(dfg_day_plot, dfg['Mag'], color = colors[2], label='ASAS-SN g-band', alpha = 0.6)
ax[0].scatter(dfv_day_plot, dfv['Mag'], color = colors[6], label = 'ASAS-SN V-band', alpha = 0.6, marker = 'v')

#ax[0].set_ylim(12.8,13.5)
ax[0].invert_yaxis() 
ax[0].set_ylabel('ASAS-SN', fontsize=20)

ax[1] = plotparams(ax[1])

ax[1].scatter(wise_day_plot, wise_w1, color=colors[1], label='W1', alpha=0.6,  marker = 's')
ax[1].scatter(wise_day_plot, wise_w2, color=colors[0], label='W2', alpha=0.6,  marker = 'x')

ax[1].invert_yaxis()
ax[1].set_ylabel('WISE', fontsize=20)
ax[1].set_xlabel('Julian Date - $2458000$ [d]', fontsize=20)

#handles1, labels1 = ax[0].get_legend_handles_labels()
#handles2, labels2 = ax[1].get_legend_handles_labels()
#handles = handles1 + handles2
#labels = labels1 + labels2
#fig.legend(handles, labels, loc='upper left', edgecolor='black', fontsize=12, bbox_to_anchor=(0.12,0.88))
plt.show()